# Strcutured output using beta client

This cookbook shows how to use beta client to get the structured output. This beta client provided a parse helper to provide your own Pydantic model instead of having to define the JSON schema and is recommended approach for supported models. Currently this supported only for gpt-4o-mini , gpt-4o-2024-08-06 models

Let's define a simple message type that carries explanation and output

In [1]:
from pydantic import BaseModel

class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

Create client and specify you want to use the beta client by specifying `use_beta_client=True`

In [2]:
import os
# Set the environment variable
os.environ['AZURE_OPENAI_ENDPOINT'] = "https://YOUR_ENDPOINT_DETAILS.openai.azure.com/"
os.environ['AZURE_OPENAI_API_KEY'] = "YOUR_API_KEY"
os.environ['AZURE_OPENAI_DEPLOYMENT_NAME'] = "gpt-4o-2024-08-06"
os.environ['AZURE_OPENAI_API_VERSION'] = "2024-08-01-preview"

In [3]:
import os
from autogen_core.components.models import AzureOpenAIChatCompletionClient, UserMessage
import json
client = AzureOpenAIChatCompletionClient(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    use_beta_client=True,
    model_capabilities={
        "vision": False,
        "function_calling": True,
        "json_output": True,
    },
)

In [6]:
messages = [
    UserMessage(content="What is the 16+32 ?", source="user"),
]
# Call the create method and pass the messages and extra_create_args as arguments 

response = await client.create(messages=messages, extra_create_args={"response_format": MathReasoning})

# Print the response
print(json.loads(response.content))

# Validate the response with the model
MathReasoning.model_validate(json.loads(response.content))

{'steps': [{'explanation': 'We start with adding the numbers 16 and 32.', 'output': '16 + 32'}, {'explanation': 'Add the ones place: 6 + 2 = 8.', 'output': '8 (ones place)'}, {'explanation': 'Add the tens place: 1 + 3 = 4.', 'output': '4 (tens place)'}, {'explanation': 'Combine the results from ones and tens places: 48.', 'output': '48'}], 'final_answer': '48'}


MathReasoning(steps=[Step(explanation='We start with adding the numbers 16 and 32.', output='16 + 32'), Step(explanation='Add the ones place: 6 + 2 = 8.', output='8 (ones place)'), Step(explanation='Add the tens place: 1 + 3 = 4.', output='4 (tens place)'), Step(explanation='Combine the results from ones and tens places: 48.', output='48')], final_answer='48')